In [173]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import lxml
import requests
from geopy.geocoders import Nominatim 
import requests 
import folium 
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
from pandas.io.json import json_normalize 

In [174]:
# Define Foursquare Credentials and Version
CLIENT_ID = '4FEBBIZLGY151WKCSXOPKA2SFEOS4AEWSCU1JVP5B32BSBT2' 
CLIENT_SECRET = '5UK5DKZRISPLDV35OP2BNVYWA3EB5LXYOJ324LFMTH21PNOB'
VERSION = '20200202'
LIMIT = 12 
radius = 500

# Data preparation and API usage

In [175]:
cities = ["New York, NY", 'Toronto, ON']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d11b941735")
    results[city] = requests.get(url).json()

Extrapolating venues locations

In [176]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

Showing locations on maps

In [177]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='red',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  

In [178]:
#New York
maps[cities[0]]

In [179]:
#Toronto
maps[cities[1]]

### At first glance, New York seemed to be more preferable city for pub crawl

In [180]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="blue", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Centre of the crawl", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))

New York, NY
Mean Distance from Mean coordinates
0.023350931101562958
Toronto, ON
Mean Distance from Mean coordinates
0.03566118620600142


In [181]:
maps[cities[0]]

In [182]:
maps[cities[1]]

#### Overall, it can be seen that for Christmas pub crawl New York will be suitable based on the needed walking avarage distance. However, for crawls with less pubs, Toronto can be a better option as a starting city. Thus, our initial hypothesis about city preference is confirmed.

# Thank you for your attention!